In [76]:
import pandas as pd
import numpy as np

In [77]:
#function to combine picks and point values
def combine(df1):
    my_file = df1
    rows = my_file.shape[0]
    i = 2 
    while i < rows:
        for col in my_file.loc[:, my_file.columns != 'Game']:
            my_file[col][i+1] = float(my_file[col][i+1])
            my_file[col][i] = [my_file[col][i], my_file[col][i+1]]
        i = i+2
    
    df1.drop([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41], axis=0, inplace=True)
    return my_file

#function to calculate totals
def totals(df):
    i = 4
    df['Score'] = 0
    df['Game Count'] = 0
    df['Max Score'] = 136
    while i < 41:
        for col in df.loc[:, ~df.columns.isin(['Game', 'Score', 'Game Count', 'Max Score'])]:
            if df[col][0] is np.nan:
                break
            
            elif df[col][0] == df[col][i][0]:
                df.loc[i, 'Game Count'] = df.loc[i, 'Game Count'] + 1
                df.loc[i, 'Score'] = df.loc[i, 'Score'] + df.loc[i, col][1]
            
            else:  
                df.loc[i, 'Max Score'] = df.loc[i, 'Max Score'] - df.loc[i, col][1]
               
        i = i + 2
    return df

#upset counter
def upset_counter(df):
    upset_count = 0
    fav_count = 0
    tie_count = 0
    for col in df.loc[:, ~df.columns.isin(['Game', 'Score', 'Max Score'])]:
        if df[col][3] < 0 and df[col][1] == df[col][0]: #if home team is favorite and away team won
            upset_count = upset_count + 1
            #print('Road upset alert ', df[col][0], "spread ", -(df[col][3]))
        elif df[col][3] > 0 and df[col][2][0] == df[col][0]: #if away team is favorite and home team won
            upset_count = upset_count + 1
            #print('Home dog win ', df[col][2])
        elif df[col][0] == "TIE":
            tie_count = tie_count + 1
            #print("A friggen tie...  ", df[col][2])
        else:
            fav_count = fav_count + 1
    print("Total of ", upset_count, "upsets this week")
    return upset_count, fav_count, tie_count


In [78]:
week1_raw = pd.read_csv("week1_results.csv")
week2_raw = pd.read_csv("Week2_results.csv")
week3_raw = pd.read_csv("week3_results.csv")
week4_raw = pd.read_csv("week4_results.csv")
week5_raw = pd.read_csv("Week5_results.csv")

week1 = combine(week1_raw)
week2 = combine(week2_raw)
week3 = combine(week3_raw)
week4 = combine(week4_raw)
week5 = combine(week5_raw)

week1_tot = totals(week1)
week2_tot = totals(week2)
week3_tot = totals(week3)
week4_tot = totals(week4)
week5_tot = totals(week5)


week1_stats = week1_tot[['Game', 'Score', 'Game Count']][4:]
week2_stats = week2_tot[['Game', 'Score', 'Game Count']][4:]
week3_stats = week3_tot[['Game', 'Score', 'Game Count']][4:]
week4_stats = week4_tot[['Game', 'Score', 'Game Count']][4:]
week5_stats = week5_tot[['Game', 'Score', 'Game Count', 'Max Score']][4:]

In [81]:
week5_stats['Week Rank'] = week5_stats['Score'].rank(ascending=False)
week5_stats.sort_values(['Score'], ascending=False).head()

,Game,Score,Game Count,Max Score,Week Rank
40,Zapp,96,11,110,1.0
36,UrBoiJZ,95,10,105,2.0
14,Ken,94,11,117,3.0
24,Michael Zwartz,87,9,92,4.0
6,Bob Z,86,11,105,5.0


In [80]:
#upset count week 1
upset_count1 = upset_counter(week1)
upset_count2 = upset_counter(week2)
upset_count3 = upset_counter(week3)
upset_count4 = upset_counter(week4)
upset_count5 = upset_counter(week5)


Total of  6 upsets this week
Total of  5 upsets this week
Total of  9 upsets this week
Total of  5 upsets this week
Total of  4 upsets this week


In [28]:
#find season totals 
season_totals = week1_stats[['Score', 'Game Count']] + week2_stats[['Score', 'Game Count']] + week3_stats[['Score', 'Game Count']] + week4_stats[['Score', 'Game Count']] + week5_stats[['Score', 'Game Count']]
season_totals = season_totals.rename(columns={'Score': 'Season Points Total', 'Game Count': 'Season Total Game Count'})

season_totals = season_totals.set_index(week1_stats['Game'])
season_totals.sort_values(['Season Total Game Count'], ascending=False).head()

,Season Points Total,Season Total Game Count
Game,,
Bob Z,395,52
Zapp,395,48
Lindsey,390,48
Kathy Zwartz,392,47
Ellieisawesome1,353,44
